# Imports

In [32]:
import time
from quasi_newton_method import quasi_newton_method_1D
from secant_method import secant_method_1D
import numpy as np
from Rosenbrok_Powell import rosenbrock_grad, powell_quartic_grad, powell_quartic_1d, powell_quartic_1d_grad, rosenbrock_1d, rosenbrock_1d_grad
import math
import sympy as sp

# Helper functions & constants

### Constants

In [33]:
a = 0
b = 3
interval = [a,b]
EPSILON = 0.01
benchMarks = {}

### Functions

In [34]:
def approximate_gradient(func, x, h=1e-5):
    """Approximates the gradient of a function using finite differences."""
    grad = np.zeros_like(x, dtype=float)  # Ensure float type
    for i in range(len(x)):
        x_plus_h = x.copy()
        x_plus_h[i] += h
        grad[i] = (func(x_plus_h) - func(x)) / h
    return grad

In [35]:
# 1D minimization problem
def f_1d_problem(x):
    return 2 * (x)**2 - 5 * x + 3

# Fibonacci

## Initialization

In [36]:
Fs = [1,1]
maxFibonacciNumber = (interval[1]-interval[0])/EPSILON
while Fs[-1] < maxFibonacciNumber:
    Fs.append(Fs[-1] + Fs[-2])
Fs.pop()
Fs

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233]

## Setting Initial points

In [37]:
k = len(Fs)

x1 = a + (Fs[k-3] / Fs[k-1]) * (b - a)
x2 = a + (Fs[k-2] / Fs[k-1]) * (b - a)

ITERATIONS = k - 2

## Computing x1 and x2 in F(X)

In [38]:
fibonacciStartTime = time.time()
fibonacciIterations = 0
for i in range(ITERATIONS):
    fibonacciIterations+=1
    newInterval = []
    f_X1 = 2 * (x1)**2 - 5 * x1 + 3
    f_X2 = 2 * (x2)**2 - 5 * x2 + 3
    if f_X1 > f_X2:
        a = x1
        x1 = x2 
        x2 = a+ (Fs[k-2]/Fs[k-1]) * (b-a)
    else:
        b = x2
        x2 = x1
        x1 = a + (Fs[k-3]/Fs[k-1]) * (b-a)
    newInterval.append(round(a,3))
    newInterval.append(round(b,3))
    print(newInterval)
    k-=1
    if b-a <= EPSILON:
        
        break
fibonacciEndTime = time.time()

[0, 1.854]
[0.708, 1.854]
[0.708, 1.416]
[0.979, 1.416]
[1.146, 1.416]
[1.146, 1.313]
[1.21, 1.313]
[1.21, 1.273]
[1.233, 1.273]
[1.233, 1.257]
[1.241, 1.257]


## Results

In [39]:
finalInterval = []
finalInterval.append(round(a,3))
finalInterval.append(round(b,3))
print(finalInterval)
fibonacciMinimum = (a+b) / 2
print(round(fibonacciMinimum,3))
fibonacciF_min = 2 * (fibonacciMinimum)**2 - 5 * fibonacciMinimum + 3
print(round(fibonacciF_min,3))

[1.241, 1.257]
1.249
-0.125


## Benchmark

In [40]:
print(f"Converged after {fibonacciIterations} iterations!")
fibonacciTakenTime = fibonacciEndTime - fibonacciStartTime
print(f"Taken: {round(fibonacciTakenTime,9)} Seconds")

Converged after 11 iterations!
Taken: 0.000998735 Seconds


## Adding to benchmark to compare later

In [41]:
benchMarks["Fibonacci"] = {
                           "Iterations":fibonacciIterations,
                           "TimeTaken": round(fibonacciTakenTime, 6),
                           "F": round(fibonacciF_min, 3),
                           "X": round(fibonacciMinimum, 3)
                           }

# Golden Section

## Initialize interior Points

In [42]:
GOLDENRATIO = (math.sqrt(5)-1) / 2
x1 = a + (1-GOLDENRATIO) * (b - a)
x2 = a + GOLDENRATIO * (b - a)

## Substitute in function

In [43]:
f_X1 = 2 * (x1)**2 - 5 * x1 + 3
f_X2 = 2 * (x2)**2 - 5 * x2 + 3
goldenStartTime = time.time()
goldenIterations = 0
while (b - a) > EPSILON:
    goldenIterations+=1
    currentInterval = []
    if f_X1  < f_X2:
        b = x2
        x2 = x1
        x1 = a + (1-GOLDENRATIO) * (b-a)
    elif f_X1 > f_X2:
        a = x1
        x1 = x2
        x2 = a + GOLDENRATIO * (b-a)
    f_X1 = 2 * (x1)**2 - 5 * x1 + 3
    f_X2 = 2 * (x2)**2 - 5 * x2 + 3
    currentInterval.append(round(a,3))
    currentInterval.append(round(b,3))
    print(currentInterval)

goldenEndTime = time.time()


[1.247, 1.257]


## Results

In [44]:
goldenMinimum = (a+b) / 2
print(round(goldenMinimum,3))
goldenF_min = 2 * (goldenMinimum)**2 - 5 * goldenMinimum + 3
print(round(goldenF_min,3))

1.252
-0.125


## Benchmark

In [45]:
print(f"Converged after {goldenIterations} iterations")
goldenTakenTime = goldenEndTime - goldenStartTime
print(f"Taken: {round(goldenTakenTime,9)} Seconds")

Converged after 1 iterations
Taken: 0.0 Seconds


## Adding to benchmark to compare later

In [46]:
benchMarks["Golden"] = {
                        "Iterations":goldenIterations,
                        "TimeTaken": round(goldenTakenTime, 6),
                        "F": round(goldenF_min, 3),
                        "X": round(goldenMinimum, 3)
                        }


# Newton

## Initial X and Computing 1st and 2nd derivatives

In [47]:
xSymbol = sp.symbols('x')
newtonF_x = 2 * (xSymbol)**2 - 5 * xSymbol + 3
dF_x_1 = sp.diff(newtonF_x, xSymbol)
dF_x_2 = sp.diff(dF_x_1, xSymbol)

## Updating x

In [48]:
newtonX = 2
newtonOldX = 0
newtonIterations = 0
# x = x - (dF_x_1/dF_x_2)
newtonStartTime = time.time()
while abs(newtonX-newtonOldX) > EPSILON:
    newtonIterations+=1
    dF_x_1_atx = dF_x_1.subs(xSymbol,newtonX)
    dF_x_2_atx = dF_x_2.subs(xSymbol,newtonX)
    newtonOldX = newtonX
    newtonX = float(newtonX - (dF_x_1_atx/dF_x_2_atx))
    print(newtonX)
    if newtonX-newtonOldX < EPSILON:
        
        break
    
newtonEndTime = time.time()

1.25


## Results

In [49]:
print(newtonX)
newtonF_min = 2*(newtonX)**2 -5 * newtonX +3
print(newtonF_min)

1.25
-0.125


## Benchmark

In [50]:
print(f"Converged after {newtonIterations} iterations")
newtonTakenTime = newtonEndTime - newtonStartTime
print(f"Taken: {newtonTakenTime} Seconds")

Converged after 1 iterations
Taken: 0.0 Seconds


## Adding to benchmark to compare later

In [51]:
benchMarks["Newton"] = {
                        "Iterations":newtonIterations,
                        "TimeTaken": round(newtonTakenTime, 6),
                        "F": round(newtonF_min, 3),
                        "X": round(newtonX, 3)
                        }

In [52]:
benchMarks

{'Fibonacci': {'Iterations': 11,
  'TimeTaken': 0.000999,
  'F': -0.125,
  'X': 1.249},
 'Golden': {'Iterations': 1, 'TimeTaken': 0.0, 'F': -0.125, 'X': 1.252},
 'Newton': {'Iterations': 1, 'TimeTaken': 0.0, 'F': -0.125, 'X': 1.25}}

# Refractoring Fibonacci, Golden Section and Newton methods into functions (Modularization)

### Fibonacci

In [53]:
def fibonacci_method_1d(f, interval, epsilon):
    Fs = [1, 1]
    max_fibonacci_number = (interval[1] - interval[0]) / epsilon
    while Fs[-1] < max_fibonacci_number:
        Fs.append(Fs[-1] + Fs[-2])
    Fs.pop()
    k = len(Fs)

    a, b = interval
    x1 = a + (Fs[k - 3] / Fs[k - 1]) * (b - a)
    x2 = a + (Fs[k - 2] / Fs[k - 1]) * (b - a)

    iterations = 0
    start_time = time.time()
    for i in range(k - 2):
        iterations += 1
        f_x1 = f(x1)
        f_x2 = f(x2)
        if f_x1 > f_x2:
            a = x1
            x1 = x2
            x2 = a + (Fs[k - 2] / Fs[k - 1]) * (b - a)
        else:
            b = x2
            x2 = x1
            x1 = a + (Fs[k - 3] / Fs[k - 1]) * (b - a)
        k -= 1
        if b - a <= epsilon:
            break
    end_time = time.time()

    minimum = (a + b) / 2
    f_min = f(minimum)
    cpu_time = end_time - start_time
    return minimum, f_min, iterations, cpu_time

### Golden Section

In [54]:
def golden_section_method_1d(f, interval, epsilon):
    golden_ratio = (math.sqrt(5) - 1) / 2
    a, b = interval
    x1 = a + (1 - golden_ratio) * (b - a)
    x2 = a + golden_ratio * (b - a)

    f_x1 = f(x1)
    f_x2 = f(x2)

    iterations = 0
    start_time = time.time()
    while (b - a) > epsilon:
        iterations += 1
        if f_x1 < f_x2:
            b = x2
            x2 = x1
            x1 = a + (1 - golden_ratio) * (b - a)
        elif f_x1 > f_x2:
            a = x1
            x1 = x2
            x2 = a + golden_ratio * (b - a)
        f_x1 = f(x1)
        f_x2 = f(x2)

    end_time = time.time()
    minimum = (a + b) / 2
    f_min = f(minimum)
    cpu_time = end_time - start_time
    return minimum, f_min, iterations, cpu_time

### Newton

In [55]:
def newtons_method_1d(f, f_prime, f_double_prime, x0, tol=1e-5, max_iter=1000):
    x = x0
    old_x = 0
    iterations = 0
    start_time = time.time()

    while abs(x - old_x) > tol and iterations < max_iter:
        iterations += 1
        first_deriv = f_prime(x)
        second_deriv = f_double_prime(x)
        old_x = x
        x = x - (first_deriv/second_deriv)
    end_time = time.time()
    f_min = f(x)
    cpu_time = end_time - start_time
    return x, f_min, iterations, cpu_time

# Quasi-Newton Method (1D)

In [56]:
def quasi_newton_method_1d(f, f_prime, x0, tol=1e-5, max_iter=1000, h=1e-5):
    x = x0
    iterations = 0
    start_time = time.time()

    # Approximate Hessian using finite difference
    hessian = (f_prime(x + h) - f_prime(x))/h

    while abs(f_prime(x)) > tol and iterations < max_iter:
        x = x - f_prime(x) / hessian  # Update x based on approximated Hessian
        iterations += 1
    end_time = time.time()
    cpu_time = end_time - start_time
    return x, f(x), iterations, cpu_time

# Secant Method (1D)

In [57]:
def secant_method_1d(f, x0, x1, tol=1e-5, max_iter=1000):
    start_time = time.time()
    x_prev = x0
    x_curr = x1
    iterations = 0
    while abs(f(x_curr) - f(x_prev)) > tol and iterations < max_iter:
        x_next = x_curr - f(x_curr) * (x_curr - x_prev) / \
            (f(x_curr) - f(x_prev))  # Secant formula
        x_prev = x_curr
        x_curr = x_next
        iterations += 1
    end_time = time.time()
    cpu_time = end_time - start_time
    return x_curr, f(x_curr), iterations, cpu_time

# Benchmark Functions

### Rosenbrock's function and its gradient

In [58]:
def rosenbrock(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2


def rosenbrock_grad(x):
    return np.array([
        -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0]),
        200 * (x[1] - x[0]**2)
    ])

### Powell's quartic function

In [59]:
def powell_quartic(x):
    return (x[0] + 10 * x[1])**2 + 5 * (x[2] - x[3])**2 + (x[1] - 2 * x[2])**4 + 10 * (x[0] - x[3])**4

# For Powell's quartic, finding the gradient analytically is complex; let's use finite differences


def powell_quartic_grad(x, h=1e-5):
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x_plus_h = x.copy()
        x_plus_h[i] += h
        grad[i] = (powell_quartic(x_plus_h) - powell_quartic(x)) / h
    return grad

### 1D minimization with Line Search using Rosenbrock Function

In [60]:
def rosenbrock_1d(x, d, alpha):
    x_new = x + alpha*d
    return rosenbrock(x_new)


def rosenbrock_1d_grad(x, d, alpha):
    h = 1e-5
    return (rosenbrock_1d(x, d, alpha+h)-rosenbrock_1d(x, d, alpha))/h

### 1D minimization with Line Search using Powell's Function

In [61]:
def powell_quartic_1d(x, d, alpha):
    x_new = x + alpha*d
    return powell_quartic(x_new)


def powell_quartic_1d_grad(x, d, alpha):
    h = 1e-5
    return (powell_quartic_1d(x, d, alpha+h)-powell_quartic_1d(x, d, alpha))/h

# Main

In [62]:
bench_marks = {}

## 1D Minimization Tests


In [83]:
print("--- 1D Minimization on f(x) = 2x^2 - 5x + 3 ---")
# -- Fibonacci Method --
minimum, f_min, iterations, cpu_time = fibonacci_method_1d(
    f_1d_problem, interval, EPSILON)
print(f"Fibonacci Method - Min: {minimum:.3f}, f(min): {
      f_min:.3f}, Iterations: {iterations}, Time: {cpu_time:.6f}")
bench_marks["Fibonacci"] = {"Iterations": iterations, "TimeTaken": round(
    cpu_time, 6), "F": round(f_min, 3), "X": round(minimum, 3)}

# -- Golden Section Method --
minimum, f_min, iterations, cpu_time = golden_section_method_1d(
    f_1d_problem, interval, EPSILON)
print(f"Golden Section Method - Min: {minimum:.3f}, f(min): {
      f_min:.3f}, Iterations: {iterations}, Time: {cpu_time:.6f}")
bench_marks["Golden"] = {"Iterations": iterations, "TimeTaken": round(
    cpu_time, 6), "F": round(f_min, 3), "X": round(minimum, 3)}

# -- Newton's Method --
xSymbol = sp.symbols('x')
newtonF_x = 2 * (xSymbol)**2 - 5 * xSymbol + 3
dF_x_1 = sp.lambdify(xSymbol, sp.diff(newtonF_x, xSymbol))
dF_x_2 = sp.lambdify(xSymbol, sp.diff(
    sp.diff(newtonF_x, xSymbol), xSymbol))

minimum, f_min, iterations, cpu_time = newtons_method_1d(
    f_1d_problem, dF_x_1, dF_x_2, 2, tol=EPSILON)
print(f"Newton's Method - Min: {minimum:.3f}, f(min): {
      f_min:.3f}, Iterations: {iterations}, Time: {cpu_time:.6f}")
bench_marks["Newton"] = {"Iterations": iterations, "TimeTaken": round(
    cpu_time, 6), "F": round(f_min, 3), "X": round(minimum, 3)}

--- 1D Minimization on f(x) = 2x^2 - 5x + 3 ---
Fibonacci Method - Min: 1.249, f(min): -0.125, Iterations: 11, Time: 0.000000
Golden Section Method - Min: 1.248, f(min): -0.125, Iterations: 12, Time: 0.000000
Newton's Method - Min: 1.250, f(min): -0.125, Iterations: 2, Time: 0.000000


## 2D Minimization with Line Search

### Rosenbrock Function

In [85]:
print("\n--- 2D Minimization on Rosenbrock with Line Search ---")
x0_rosen = np.array([-1.2, 1.0], dtype=float)
d_rosen = -rosenbrock_grad(x0_rosen)
initial_alpha = 0.0
initial_alpha_secant = 0.1
interval_rosen = [0, 3]

# Fibonacci
min_alpha, min_val, iterations, cpu_time = fibonacci_method_1d(
    lambda alpha: rosenbrock_1d(x0_rosen, d_rosen, alpha), interval_rosen, EPSILON)
print(f"Fibonacci (Rosenbrock) - Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, CPU time: {cpu_time:.6f}")

# Golden Section
min_alpha, min_val, iterations, cpu_time = golden_section_method_1d(
    lambda alpha: rosenbrock_1d(x0_rosen, d_rosen, alpha), interval_rosen, EPSILON)
print(f"Golden Section (Rosenbrock) - Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, CPU time: {cpu_time:.6f}")

# Quasi-Newton
min_alpha, min_val, iterations, cpu_time = quasi_newton_method_1d(
    lambda alpha: rosenbrock_1d(x0_rosen, d_rosen, alpha),
    lambda alpha: rosenbrock_1d_grad(x0_rosen, d_rosen, alpha),
    initial_alpha
)
print(f"Quasi-Newton (Rosenbrock) - Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, CPU time: {cpu_time:.6f}")

# Secant
min_alpha, min_val, iterations, cpu_time = secant_method_1d(
    lambda alpha: rosenbrock_1d(x0_rosen, d_rosen, alpha),
    initial_alpha, initial_alpha_secant
)
print(f"Secant (Rosenbrock)- Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, CPU time: {cpu_time:.6f}")

# Newton
xSymbol = sp.symbols('alpha')
rosenbrock_x = rosenbrock_1d(x0_rosen, d_rosen, xSymbol)
dF_x_1 = sp.lambdify(xSymbol, sp.diff(rosenbrock_x, xSymbol))
dF_x_2 = sp.lambdify(xSymbol, sp.diff(
    sp.diff(rosenbrock_x, xSymbol), xSymbol))

min_alpha, min_val, iterations, cpu_time = newtons_method_1d(
    lambda alpha: rosenbrock_1d(x0_rosen, d_rosen, alpha), dF_x_1, dF_x_2, 2, tol=EPSILON)
print(f"Newton (Rosenbrock) - Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, Time: {cpu_time:.6f}")


--- 2D Minimization on Rosenbrock with Line Search ---
Fibonacci (Rosenbrock) - Optimal alpha: 0.007957, f(x): 205.982138, Iterations: 11, CPU time: 0.001006
Golden Section (Rosenbrock) - Optimal alpha: 0.010417, f(x): 67.724427, Iterations: 12, CPU time: 0.000000
Quasi-Newton (Rosenbrock) - Optimal alpha: 0.000783, f(x): 4.128805, Iterations: 18, CPU time: 0.000000
Secant (Rosenbrock)- Optimal alpha: 0.012262, f(x): 0.199322, Iterations: 400, CPU time: 0.004149
Newton (Rosenbrock) - Optimal alpha: 0.022449, f(x): 10561.833050, Iterations: 12, Time: 0.000000


### Powell's Quartic Function

In [86]:
print("\n--- 2D Minimization on Powell's Quartic with Line Search ---")
x0_powell = np.array([3.0, -1.0, 0.0, 1.0], dtype=float)
d_powell = -powell_quartic_grad(x0_powell)
initial_alpha = 0.0
initial_alpha_secant = 0.1
interval_powell = [0, 3]

# Fibonacci
min_alpha, min_val, iterations, cpu_time = fibonacci_method_1d(
    lambda alpha: powell_quartic_1d(x0_powell, d_powell, alpha), interval_powell, EPSILON)
print(f"Fibonacci (Powell) - Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, CPU time: {cpu_time:.6f}")

# Golden Section
min_alpha, min_val, iterations, cpu_time = golden_section_method_1d(
    lambda alpha: powell_quartic_1d(x0_powell, d_powell, alpha), interval_powell, EPSILON)
print(f"Golden Section (Powell) - Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, CPU time: {cpu_time:.6f}")

# Quasi-Newton
min_alpha, min_val, iterations, cpu_time = quasi_newton_method_1d(
    lambda alpha: powell_quartic_1d(x0_powell, d_powell, alpha),
    lambda alpha: powell_quartic_1d_grad(x0_powell, d_powell, alpha),
    initial_alpha
)
print(f"Quasi-Newton (Powell) - Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, CPU time: {cpu_time:.6f}")

# Secant
min_alpha, min_val, iterations, cpu_time = secant_method_1d(
    lambda alpha: powell_quartic_1d(x0_powell, d_powell, alpha),
    initial_alpha, initial_alpha_secant
)
print(f"Secant (Powell)- Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, CPU time: {cpu_time:.6f}")

# Newton
xSymbol = sp.symbols('alpha')
powell_x = powell_quartic_1d(x0_powell, d_powell, xSymbol)
dF_x_1 = sp.lambdify(xSymbol, sp.diff(powell_x, xSymbol))
dF_x_2 = sp.lambdify(xSymbol, sp.diff(sp.diff(powell_x, xSymbol), xSymbol))

min_alpha, min_val, iterations, cpu_time = newtons_method_1d(
    lambda alpha: powell_quartic_1d(x0_powell, d_powell, alpha), dF_x_1, dF_x_2, 2, tol=EPSILON)
print(f"Newton (Powell) - Optimal alpha: {min_alpha:.6f}, f(x): {
      min_val:.6f}, Iterations: {iterations}, Time: {cpu_time:.6f}")


--- 2D Minimization on Powell's Quartic with Line Search ---
Fibonacci (Powell) - Optimal alpha: 0.007957, f(x): 772.643084, Iterations: 11, CPU time: 0.000000
Golden Section (Powell) - Optimal alpha: 0.004658, f(x): 38.323281, Iterations: 12, CPU time: 0.000000
Quasi-Newton (Powell) - Optimal alpha: 0.003584, f(x): 30.830347, Iterations: 693, CPU time: 0.007706
Secant (Powell)- Optimal alpha: 0.003576, f(x): 30.830704, Iterations: 76, CPU time: 0.001992
Newton (Powell) - Optimal alpha: 0.018627, f(x): 81001.037954, Iterations: 12, Time: 0.000000
